# Retrieving calibration data from Quantum Inspire

Quantum Inspire backends are calibrated periodically. It may be of interest to retrieve this calibration data from the API, for instance to be able to compare current system performance to the quoted one, verify the calibration or use the data for further computations.

This notebook shows how to retrieve calibration data for a backend and use the `CalibrationViewer` class to print it.

**Note:**

- Compilation and transpilation options not available at this moment. See knowledge base for details.
- Calibrations are coupled to experiments. Even though the calibrations are done on a periodic basis (e.g. once per day), every experiment gets mapped to such a calibration. Therefore currently we need a handle to an experiment/job to be able to retrieve calibration data, it is not yet possible to just ask for the calibration without a job reference.
- Calibrations are backend specific

**Todo:**

- Include readout error calibration in calibration data
- Pass standard deviations in calibration values (the decimals field is not suitable)
    - Related: Round numbers in calibration data view to reasonable decimals
- Include human-readable backend name ('Starmon-5' etc.) in calibration dictionary in addition to url.
    Alternatively the viewer object could parse the url into a name by asking the api for all backend descriptions (`qi_api.get_backend_types()`)

**Authors:** Pieter Eendebak <pieter.eendebak@tno.nl>, Olexiy Fedorets <o.o.fedorets@student.tudelft.nl>

In [42]:
import json
from pprint import pprint

from qiskit import execute
from qiskit.circuit import QuantumCircuit

from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI
from quantuminspire.calibration_viewer import CalibrationViewer

In [2]:
def remove_ordered_dict(d: Any) -> Any:
    """ Convert OrderedDict `d` into a python builtin dict. """
    if isinstance(d, dict):
        return {k: remove_ordered_dict(v) for k, v in d.items()}
    else:
        return d

## Set credentials

In [3]:
authentication = get_authentication()
QI.set_authentication(authentication)
qi_api = QI.get_api()

## Run an experiment to generate data and ensure calibration will be available

This step is not necessary if it is sure that an experiment already has been executed (recently! otherwise calibration data will be outdated) on the chosen backend.

In [4]:
backend_name = 'Spin-2'
if qi_api.get_backend_type_by_name(backend_name)["status"] == "OFFLINE":
        backend_name = 'Starmon-5'
qi_backend = QI.get_backend(backend_name)
print(f'Selected backend: {backend_name}')

circuit = QuantumCircuit(2)
circuit.h(1)
circuit.cx(1, 0)
circuit.measure_all()

qi_job = execute(circuit, backend=qi_backend, shots=256)
print(f'Execute job on {qi_backend}')

qi_result = qi_job.result()
histogram = qi_result.get_counts(circuit)
print(f'Measured result: {histogram}')

Selected backend: Starmon-5
Execute job on Starmon-5
Measured result: {'00': 111, '01': 13, '10': 8, '11': 124}


## Select job from backend to get calibration data from

In [5]:
qi_backend = QI.get_backend(backend_name)
all_jobs = qi_api.get_jobs()
backend_types = qi_api.get_backend_types()
backends = qi_api._get('https://api.quantum-inspire.com/backends')

backend_url = [b for b in backends if b['name']==backend_name][0]['url']
backend_jobs = [j for j in all_jobs if j['backend']==backend_url]
print(f"Found {len(backend_jobs)} jobs for backend {backend_name}.")

if len(backend_jobs)==0:
    raise Exception(f"You have not executed jobs in {backend_name}, no calibration data available.")

Found 1578 jobs for backend Starmon-5.


In [19]:
# select latest job (appears first in list of jobs)
job = backend_jobs[0]
print(f"Found job with id {job['id']}")

result = qi_api.get_result_from_job(job['id'])
calibration = qi_api.get_calibration_from_result(result['id'])
calibration = remove_ordered_dict(calibration)
pprint(calibration)
date = calibration['parameters']['system']['last_calibration_date']['value']
print(f'Last calibration data from {date}')

Found job with id 7085428
{'backend': 'https://api.quantum-inspire.com/backends/8/',
 'parameters': {'qubits': {'q0': {'initialization_fidelity': {'symbol': 'Finit',
                                                              'unit': '%',
                                                              'value': 99.8},
                                  'readout_fidelity': {'symbol': 'FR/O',
                                                       'unit': '%',
                                                       'value': 95.1},
                                  'single_qubit_gate_fidelity': {'symbol': 'F1q',
                                                                 'unit': '%',
                                                                 'value': 99.9},
                                  't1': {'decimals': 1,
                                         'symbol': 'T1',
                                         'unit': 's',
                                         'value': 1.140486376

## Create `CalibrationViewer` object to view the data

In [26]:
viewer = CalibrationViewer(calibration)
print(repr(viewer))
viewer.show_system_parameters()
viewer.show_qubit_parameters()
viewer.show_calibration_field('last_calibration_date')

try:
    c = viewer.show_calibration_field('readout_error_calibration')
except KeyError:
    print('Readout error calibration does not exist')


<calibration of backend https://api.quantum-inspire.com/backends/8/ from time 24/12/2021 - 03:49 AM>


System:

fridge_temperature: 20.9

last_calibration_date: 24/12/2021 - 03:49 AM

Qubit: q0

...

Calibration: last_calibration_date

24/12/2021 - 03:49 AM

other fields:

{'unit', 'symbol', 'value'}

Readout error calibration does not exist


The string representation of the viewer returns the calibration dictionary as a string, so we can load the calibration back into a dictionary when having access to a `CalibrationViewer` object.

In [39]:
print(viewer)

{"url": "https://api.quantum-inspire.com/calibration/109052/", "backend": "https://api.quantum-inspire.com/backends/8/", "parameters": {"system": {"fridge_temperature": {"value": "20.9", "unit": "mK", "symbol": "Fridge temperature"}, "last_calibration_date": {"value": "24/12/2021 - 03:49 AM", "unit": "iso-8601", "symbol": "Calibration date"}}, "qubits": {"q0": {"t1": {"value": 1.1404863763460713e-05, "unit": "s", "symbol": "T1", "decimals": 1}, "t2star": {"value": 1.04250551881577e-05, "unit": "s", "symbol": "T2e", "decimals": 1}, "single_qubit_gate_fidelity": {"value": 99.9, "unit": "%", "symbol": "F1q"}, "two_qubit_gate_fidelity": {"value": 97.8, "unit": "%", "symbol": "F2q"}, "initialization_fidelity": {"value": 99.8, "unit": "%", "symbol": "Finit"}, "readout_fidelity": {"value": 95.1, "unit": "%", "symbol": "FR/O"}}, "q1": {"t1": {"value": 1.3131372093163124e-05, "unit": "s", "symbol": "T1", "decimals": 1}, "t2star": {"value": 1.8620682825195337e-05, "unit": "s", "symbol": "T2e", "

In [44]:
json.loads(viewer.__str__())

{'url': 'https://api.quantum-inspire.com/calibration/109052/',
 'backend': 'https://api.quantum-inspire.com/backends/8/',
 'parameters': {'system': {'fridge_temperature': {'value': '20.9',
    'unit': 'mK',
    'symbol': 'Fridge temperature'},
   'last_calibration_date': {'value': '24/12/2021 - 03:49 AM',
    'unit': 'iso-8601',
    'symbol': 'Calibration date'}},
  'qubits': {'q0': {'t1': {'value': 1.1404863763460713e-05,
     'unit': 's',
     'symbol': 'T1',
     'decimals': 1},
    't2star': {'value': 1.04250551881577e-05,
     'unit': 's',
     'symbol': 'T2e',
     'decimals': 1},
    'single_qubit_gate_fidelity': {'value': 99.9,
     'unit': '%',
     'symbol': 'F1q'},
    'two_qubit_gate_fidelity': {'value': 97.8, 'unit': '%', 'symbol': 'F2q'},
    'initialization_fidelity': {'value': 99.8, 'unit': '%', 'symbol': 'Finit'},
    'readout_fidelity': {'value': 95.1, 'unit': '%', 'symbol': 'FR/O'}},
   'q1': {'t1': {'value': 1.3131372093163124e-05,
     'unit': 's',
     'symbol': '